In [1]:
import numpy as np
import pandas as pd

In [3]:

df_catalogue = pd.read_excel('/Users/baptistecarbillet/Documents/ConversationalAI/GT Course Catalogue.xlsx')

df_schedule = pd.read_excel('/Users/baptistecarbillet/Documents/ConversationalAI/GT Course Schedule.xlsx')

1
2


In [45]:
df_schedule_ece = df_schedule[df_schedule['CRSE_SUBJ_CODE'] == 'ECE']

In [122]:
df_schedule_ece.head()

,TERM_CODE,CRN,CRSE_SUBJ_CODE,CRSE_NUMB,CRSE_SEQ_NUMB,INSM_CODE,SCHD_CODE,START_DATE,END_DATE,BEGIN_TIME,...,TUE_DAY,WED_DAY,THU_DAY,FRI_DAY,SAT_DAY,RANK,SSRMEET_CATAGORY,SSRMEET_BLDG_CODE,BLDG_DESC,SSRMEET_MTYP_CODE
325085,202008,91712,ECE,1010,LA,P,E,2020-08-17,2020-12-10,1230.0,...,NaN,NaN,NaN,F,NaN,1,1,85,Van Leer,CLAS
325086,202008,91711,ECE,1010,A,P,A,2020-08-17,2020-12-10,930.0,...,NaN,W,NaN,NaN,NaN,2,1,85,Van Leer,CLAS
325087,202005,56856,ECE,1010,A,NaN,A,2020-06-17,2020-07-30,1100.0,...,T,NaN,R,NaN,NaN,3,1,NaN,NaN,CLAS
325088,202005,56013,ECE,1010,A01,NaN,E,2020-06-17,2020-07-30,930.0,...,NaN,W,NaN,F,NaN,4,1,NaN,NaN,CLAS
325089,201905,56651,ECE,1010,A,NaN,B,2019-06-19,2019-08-01,1005.0,...,T,NaN,R,NaN,NaN,5,1,85,Van Leer,CLAS


In [60]:
#Filter for start date in 2024

df_schedule_2024 = df_schedule_ece[df_schedule_ece['START_DATE'].dt.year == 2024]

#rename CRSE_NUMB col to CRSE_NBR

df_schedule_2024.rename(columns={'CRSE_NUMB':'CRSE_NBR'}, inplace=True)


/var/folders/_x/_fqb6r5s5rd6tk1hhyl_y7qh0000gn/T/ipykernel_87284/3954224826.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_schedule_2024.rename(columns={'CRSE_NUMB':'CRSE_NBR'}, inplace=True)


In [234]:
df_schedule_2024.shape

(1425, 23)

In [70]:
#Joibbthe two dataframes, with CRSE_NBR as key

joined_df = pd.merge(df_schedule_2024,df_ece, on='CRSE_NBR', how='left')



In [95]:
### REMOVE ROWS WITH NAN IN ALL 7 'SUN_DAY', 'MON_DAY', 'TUE_DAY', 'WED_DAY', 'THU_DAY','FRI_DAY', 'SAT_DAY'
new_df = joined_df.dropna(subset=['SUN_DAY', 'MON_DAY', 'TUE_DAY', 'WED_DAY', 'THU_DAY','FRI_DAY', 'SAT_DAY'], how='all')


In [102]:
new_df.iloc[:,0:10]

,TERM_CODE,CRN,CRSE_SUBJ_CODE,CRSE_NBR,CRSE_SEQ_NUMB,INSM_CODE,SCHD_CODE,START_DATE,END_DATE,BEGIN_TIME
0,202402,29812,ECE,1100,A,NaN,S,2024-01-08,2024-05-02,1700.0
1,202402,29183,ECE,2002,A,NaN,D,2024-01-08,2024-05-02,1230.0
2,202402,30667,ECE,2020,D,NaN,A,2024-01-08,2024-05-02,1400.0
3,202402,26982,ECE,2020,A,NaN,A,2024-01-08,2024-05-02,1100.0
4,202402,25939,ECE,2020,C,NaN,A,2024-01-08,2024-05-02,1230.0
...,...,...,...,...,...,...,...,...,...,...
767,202402,30707,ECE,8803,ECS,NaN,A,2024-01-08,2024-05-02,1700.0
768,202402,29757,ECE,8803,R,NaN,A,2024-01-08,2024-05-02,930.0
769,202402,33390,ECE,8804,VLS,NaN,A,2024-01-08,2024-05-02,1230.0
774,202402,33349,ECE,8893,AML,NaN,A,2024-01-08,2024-05-02,1700.0


In [199]:
l = []
for title in new_df.CRSE_TITLE_LONG.unique():
    row = new_df[new_df['CRSE_TITLE_LONG'] == title]
    l.append(row.shape[0])
    if row.shape[0] == 3:
        print(title)
print(np.unique(l, return_counts=True))

Senior Analog Electronics Laboratory
Optical Fiber Communications
Statistical Machine Learning
(array([ 0,  1,  2,  3,  6,  7, 10, 15, 16, 18, 19]), array([ 1, 47, 12,  3,  1,  1,  1,  1,  2,  1,  1]))


In [225]:
def createQuestionDate(class_name):
    row = new_df[new_df['CRSE_TITLE_LONG'] == class_name]
    #print(row.shape)
    if row.shape[0] == 0:
        chat = {
        "instruction": "When is taught " + class_name + "?",
        "input": "",
        "output": "Sorry, I don't know when " + class_name + " is taught."}
        
        return chat
    #print(row.loc[:,['SUN_DAY', 'MON_DAY', 'TUE_DAY', 'WED_DAY', 'THU_DAY','FRI_DAY', 'SAT_DAY']])
    
    
    if row.shape[0] ==1:
        #row = row.loc[0]
        days = ['SUN_DAY', 'MON_DAY', 'TUE_DAY', 'WED_DAY', 'THU_DAY','FRI_DAY', 'SAT_DAY']
        match_day = {}
        match_day['SUN_DAY'] = 'Sunday'
        match_day['MON_DAY'] = 'Monday'
        match_day['TUE_DAY'] = 'Tuesday'
        match_day['WED_DAY'] = 'Wednesday'
        match_day['THU_DAY'] = 'Thursday'
        match_day['FRI_DAY'] = 'Friday'
        match_day['SAT_DAY'] = 'Saturday'
        day_list = []
        for day in days:
            if not row[day].isna().values[0]:
                day = match_day[day]
                day_list.append(day)
        #print(len(day_list))
        if len(day_list) == 0:
            return  {
        "instruction": "When is taught " + class_name + "?",
        "input": "",
        "output": "Sorry, I don't know when " + class_name + " is taught."}
            
        if len(day_list) == 2:
            
            begin_time = str(row['BEGIN_TIME'].values[0])[:-2]
            end_time = str(row['END_TIME'].values[0])[:-2]
            
            bt = begin_time[:-2] + ':' + begin_time[-2:]
            et = end_time[:-2] + ':' + end_time[-2:]
            
            chat = {
            "instruction": "When is taught " + class_name + "?",
            "input": "",
            "output": "It is taught on " + day_list[0] + " and " + day_list[1] + " from " + bt + " to " + et}
            
        if len(day_list) == 1:
            
            begin_time = str(row['BEGIN_TIME'].values[0])[:-2]
            end_time = str(row['END_TIME'].values[0])[:-2]
            
            bt = begin_time[:-2] + ':' + begin_time[-2:]
            et = end_time[:-2] + ':' + end_time[-2:]
            
            chat = {
            "instruction": "When is taught " + class_name + "?",
            "input": "",
            "output": "It is taught on " + day_list[0] + " from " + bt + " to " + et}
            
        if len(day_list) == 3:
            
            begin_time = str(row['BEGIN_TIME'].values[0])[:-2]
            end_time = str(row['END_TIME'].values[0])[:-2]
            
            bt = begin_time[:-2] + ':' + begin_time[-2:]
            et = end_time[:-2] + ':' + end_time[-2:]
            
            chat = {
            "instruction": "When is taught " + class_name + "?",
            "input": "",
            "output": "It is taught on " + day_list[0] + ", " + day_list[1] + " and " + day_list[2] + " from " + bt + " to " + et}
            
                
        
    
    return chat

createQuestionDate('Radar Imaging')

{'instruction': 'When is taught Radar Imaging?',
 'input': '',
 'output': 'It is taught on Monday and Wednesday from 8:00 to 9:15'}

In [226]:
question_list = []
for title in new_df.CRSE_TITLE_LONG.unique():
    
    row = new_df[new_df['CRSE_TITLE_LONG'] == title]
    if row.shape[0] == 1:
        question_list.append(createQuestionDate(title))


In [261]:
## Generate compatibility questions:
def courseSchedule(title):
    row = new_df[new_df['CRSE_TITLE_LONG'] == title]
    days = ['SUN_DAY', 'MON_DAY', 'TUE_DAY', 'WED_DAY', 'THU_DAY','FRI_DAY', 'SAT_DAY']
    match_day = {}
    match_day['SUN_DAY'] = 'Sunday'
    match_day['MON_DAY'] = 'Monday'
    match_day['TUE_DAY'] = 'Tuesday'
    match_day['WED_DAY'] = 'Wednesday'
    match_day['THU_DAY'] = 'Thursday'
    match_day['FRI_DAY'] = 'Friday'
    match_day['SAT_DAY'] = 'Saturday'
    day_list = []
    for day in days:
        if not row[day].isna().values[0]:
            day = match_day[day]
            day_list.append(day)

    begin_time = str(row['BEGIN_TIME'].values[0])[:-2]
    end_time = str(row['END_TIME'].values[0])[:-2]
    
    bt = begin_time[:-2] + ':' + begin_time[-2:]
    et = end_time[:-2] + ':' + end_time[-2:]
    
    return [(day,bt,et) for day in day_list]


schedule_list = {}

for title in new_df.CRSE_TITLE_LONG.unique():
    
    row = new_df[new_df['CRSE_TITLE_LONG'] == title]
    if row.shape[0] == 1:
        schedule_list[title] = courseSchedule(title)

In [257]:
from datetime import datetime, timedelta

def is_time_conflict(course1_schedule, course2_schedule):
    # Parse day and time information for each course
    day1, start_time1, end_time1 = course1_schedule
    day2, start_time2, end_time2 = course2_schedule

    # Define a mapping from day strings to numerical values
    days_mapping = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4}

    # Convert day strings to numerical values
    day_num1 = days_mapping[day1]
    day_num2 = days_mapping[day2]

    # Define time format and convert start and end times to datetime objects
    time_format = "%H:%M"
    start_datetime1 = datetime.strptime(start_time1, time_format)
    end_datetime1 = datetime.strptime(end_time1, time_format)
    start_datetime2 = datetime.strptime(start_time2, time_format)
    end_datetime2 = datetime.strptime(end_time2, time_format)

    # Check if the days overlap
    if day_num1 == day_num2:
        # Check if there is a time overlap
        if start_datetime1 < end_datetime2 and start_datetime2 < end_datetime1:
            return True

    # No overlap
    return False

# Example usage
course1_schedule = ('Monday', '14:00', '15:15')
course2_schedule = ('Monday', '14:30', '16:00')

if is_time_conflict(course1_schedule, course2_schedule):
    print("There is a time conflict between the two courses.")
else:
    print("No time conflict between the two courses.")

There is a time conflict between the two courses.


In [276]:
def generateString(title,schedule):
    
    n = len(schedule)
    hour_string = " from {} to {}".format(schedule[0][1],schedule[0][2])
    day_string = schedule[0][0]
    for i in range(1,n):
        day_string += ' and {}'.format(schedule[i][0])
        
    return title +' is taught ' +day_string + hour_string

#generateString(schedule_list)
    

In [275]:
generateString('Advanced Digital Signal Processing',schedule_list['Advanced Digital Signal Processing'])

'Advanced Digital Signal Processing is taught Tuesday and Thursday from 8:00 to 9:15.'

In [267]:
schedule_list['Advanced Digital Signal Processing']

[('Tuesday', '8:00', '9:15'), ('Thursday', '8:00', '9:15')]

In [289]:
def generateCompatibilty(title1,title2):
    
    schedule1 = schedule_list[title1]
    schedule2 = schedule_list[title2]
    
    compatibility = True
    for s1 in schedule1:
        for s2 in schedule2:
            
            compatibility = compatibility and not(is_time_conflict(s1,s2))
            
            
            
    if compatibility:
        
        chat =  {'instruction': 'Can I take {} and {}'.format(title1,title2),
    'input': '',
    'output': 'Yes, ' + generateString(title1,schedule_list[title1]) + ' and ' + generateString(title2,schedule_list[title2])}
            
    else: 
        chat  =  {'instruction': 'Can I take {} and {}'.format(title1,title2),
    'input': '',
    'output': 'No the classes are overlapping, '  + generateString(title1,schedule_list[title1]) + ' and ' + generateString(title2,schedule_list[title2]) }
        
        
    return chat
            
            

In [290]:
generateCompatibilty('Robotics Research Foundation I' ,'Introduction to Computer Security')

{'instruction': 'Can I take Robotics Research Foundation I and Introduction to Computer Security',
 'input': '',
 'output': 'Yes, Robotics Research Foundation I is taught Friday from 11:00 to 12:55 and Introduction to Computer Security is taught Monday and Wednesday from 15:30 to 16:45'}

In [295]:
question_list2 = []
for title1 in schedule_list.keys():
    for title2 in schedule_list.keys():
        if title1 != title2:
            
            question_list2.append(generateCompatibilty(title1,title2))

In [296]:
#convert question_list to json and save it
import json
with open('question_list2.json', 'w') as fout:
    json.dump(question_list, fout)





In [193]:
row = new_df[new_df['CRSE_TITLE_LONG'] == 'Technology Entrepreneurship: Teaming, Ideation, Entrepreneurship, Intrepreneurship, and Leadership']
x= row.loc[:,['SUN_DAY', 'MON_DAY', 'TUE_DAY', 'WED_DAY', 'THU_DAY','FRI_DAY', 'SAT_DAY']]

In [194]:
x

,SUN_DAY,MON_DAY,TUE_DAY,WED_DAY,THU_DAY,FRI_DAY,SAT_DAY
492,NaN,NaN,NaN,W,NaN,NaN,NaN
493,NaN,NaN,NaN,NaN,R,NaN,NaN
494,NaN,NaN,NaN,NaN,NaN,F,NaN
495,NaN,NaN,NaN,NaN,NaN,F,NaN
496,NaN,NaN,NaN,NaN,R,NaN,NaN
498,NaN,NaN,NaN,W,NaN,NaN,NaN
499,NaN,NaN,NaN,W,NaN,NaN,NaN


In [213]:
createQuestionDate('Theory of Electronic Devices')

(1, 41)
    SUN_DAY MON_DAY TUE_DAY WED_DAY THU_DAY FRI_DAY SAT_DAY
540     NaN     NaN     NaN       W     NaN     NaN     NaN


{'instruction': 'When is taught Theory of Electronic Devices?',
 'input': '',
 'output': 'It is taught on Wednesday from 8:00 to 9:15'}